In [12]:
import time
import torch
import triton
import triton.language as tl
import torch.nn.functional as F

In [11]:
sample=torch.tensor(
    [
        [1,2,3],
        [5,6,7]
    ],
    dtype=torch.float32,
    device='cuda'
)

out=F.softmax(
    sample,
    dim=1
)
out

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]], device='cuda:0')

In [18]:
def softmax(
        x:torch.Tensor
)-> torch.Tensor:
    x_max=x.max(dim=1)[0]
    return x_max

In [19]:
softmax(sample)

tensor([3., 7.], device='cuda:0')